## **Dependencies and Import**

Installing Dependencies and Importing Libraries for Web Scraping 🕸 and Data Processing and Visualization 📈📊


### **Dependencies Installation 📦🛠**


In [1]:
# Data Extraction
%pip install beautifulsoup4
%pip install requests
%pip install urllib3
%pip install html5lib
# Data Manipulation
%pip install pandas


Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.


### **Import Libraries ⬇📦**


In [2]:
# Data Extraction
from bs4 import BeautifulSoup as BS
import requests
import urllib3
import json
# Data Manipulation
import pandas as pd
# File Handling
import os


## **Extracting Company Data 📦**

Useful request types to make web and api requests fro data extraction

In [3]:
# Request List
REQUESTS = ["GET", "POST", "PUT", "DELETE"]

# Request type dictionary
REQUEST_TYPE = {
    "GET": REQUESTS[0],
    "POST": REQUESTS[1],
    "PUT": REQUESTS[2],
    "DELETE": REQUESTS[3],
}


### **Data Extraction Functions 📃🕸**

Defining functions to scrape and intercepted api request's data from [NEPSE](http://www.nepalstock.com) and [SmartWealthPro](https://app.smartwealthpro.com).


#### **Data Extraction From [Nepse](http://www.nepalstock.com) Website 📃**
After reading [Quassarian Viper](https://q-viper.github.io/2020/11/21/deploying-nepse-data-visualizer-on-heroku/)'s post I found an data extraction method from  [Nepse](http://www.nepalstock.com) website and created functions respectively. 

In [4]:

def nepse_company_names(save_to_csv: bool = False) -> pd.DataFrame:
    """Extracts all the company names from NEPSE website

    Args:
        save_to_csv: Save the data to csv file - Ex: True

    Return:
        DataFrame of the company names

    """
    http = urllib3.PoolManager()
    http.headers.update({'User-Agent': 'Mozilla/5.0'})
    web_page = http.request(
        method=REQUEST_TYPE["GET"],
        url="http://www.nepalstock.com/company?_limit=500"
    )
    soup = BS(web_page.data, 'html5lib')
    table = soup.find('table')
    company = []
    rows = [row.findAll('td') for row in table.findAll('tr')[1:-2]]
    col = 0
    notfirstrun = False
    for row in rows:
        companydata = []
        for data in row:
            if col == 5 and notfirstrun:
                companydata.append(data.a.get('href').split('/')[-1])
            else:
                companydata.append(data.text.strip())
            col += 1
        company.append(companydata)
        col = 0
        notfirstrun = True

    df = pd.DataFrame(company[1:], columns=company[0])
    df.rename(columns={'Operations': 'Symbol No'}, inplace=True)
    df.drop(columns='', inplace=True)
    df.drop(columns='S.N.', inplace=True)

    if(save_to_csv):
        path = './NepseDataset'
        if not os.path.exists(path):
            os.makedirs(path)
        df.to_csv(path+'/company_list.csv', index=False)

    return df


#### **Data Extraction From [SmartWealthPro](https://app.smartwealthpro.com/) Website 📃**

Although [Data Extraction From Nepse Website](#data-extraction-from-nepalipaisa-website-📃) didn't helped much I found a way to extract data from **paid application**. Firstly, I registered for **Free Trial** on [SmartWealthPro](https://app.smartwealthpro.com) and used [Postman](https://www.postman.com/)'s **Interceptor** to intercept request and response from browser using [Postman](https://www.postman.com/)'s [Interceptor Plugin](https://chrome.google.com/webstore/detail/postman-interceptor/aicmkgpgakddgnaphhhpliifpcfhicfo) and extracted useful URL's along with cookies and imported the request code from postman to python and created respective function which would also store dataset on local drive as csv file.


In [5]:
def smart_wealth_company_list(save_to_csv=False) -> pd.DataFrame:
    """ Retrives List of Company from SmartWealthPro which includes CompanyId Defined by SmartWealthPro and also help's to export as csv file.

    Args:
        save_to_csv: (bool) - Ex : True
    Returns:
        df: (pd.DataFrame)
    """
    url = "https://app.smartwealthpro.com/api/GetAutoCompleteCompanies?_=1648523415665"
    payload = {}
    headers = {
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Microsoft Edge";v="99"',
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Content-Type': 'application/json; charset=utf-8',
        'X-Requested-With': 'XMLHttpRequest',
        'sec-ch-ua-mobile': '?0',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36 Edg/99.0.1150.55',
        'sec-ch-ua-platform': '"Windows"',
        'Cookie': '.AspNetCore.Antiforgery.c2MYB_mZqSE=CfDJ8HpU54MlOTxHvXVwP3W4FVcE04C__HY9Ao_d2SGGKq0B4N-qC9sLKRJtbZgK6GcBsxbVn836dm4Smfhp9Q3gN3xxVVM7biw_G-LDmFOmxIxvMZCBJFgCtTHT-Ndk6_0YtyL4tlMZ8oMM9E_3yJdG_Pg; .AspNetCore.Identity.Application=CfDJ8HpU54MlOTxHvXVwP3W4FVc0oGisZYo6K0aMXQbLY6l-5FOIOjclXymmiAJmZ2RPdZloVRNo7jLq7QUOh17zthijNK1IP-41anutz94fT_x7uqQf8xVe20uX6l0P7GF1-v-Qse-nyCMdFdWKuSrSLul8-olCvdpUyiwMxFgT2nYC80fe2BI2fEaD0YnlGu7u4Fyow3yg-aXDdmLeSNGE7uPZKY50MTTlayUGXxw0wRYuPVoe62Dj0b2SCaIhox6kgps-OHbu9WgcVKsdY2d5ZUCyVo1vWN2oWdpdlJw_XUbtLyOFZgTaMUUOrGNfRcMdqI_lYCza0EqnjHpiLivCQgJf7OmUYfy-NR4pULroicDY1KIzbAfrlvJUxXuJvoy-NozUIf4Kvtp8X8Bo93T0ffuA5L6iJHGBmQ5IEbmBKi2gKu-40I-WlRhF9_Vy0Md0ywkiry3_h5NlwymwAYW37hm_vrJ35WD27IfdDrMBnsaypXUfACw2Za0RDpkcQIuxOwrgyrcS0XFi-k7jbxWX2eVfeEYqtutZJZkMEqhblEpAuGNzo-u14c6Lk_INoXN3T9IHuxeP2VD8NulAPUQMcseCHeWKDe1oVfVcyQ1PtBl7si0HnjEPgpnUTy9yBU5bgqs3OEpJC9EhLlFFRshdb7FOQdwIzadK3Oqdn9ZiDdgyaFy0hVCw-z3uvdiKy3kYklzqQ3YW-3MAP4sMYqzQGOEEe-B3wMUklA3L8wNrsXDhyleeyLeYpPsx3X9ueN1QGicU7O6OdL4xqVPfmaYonmG6Z6NhoACZIyAIlIWjiWJwU5t5sRcDYC-UE9FXlI2ygg'
    }

    response = requests.request("GET", url, headers=headers, data=payload)
    SMARTWEALTH_JSON = json.loads(response.text)
    if SMARTWEALTH_JSON["responseCode"] == 200:
        df = pd.DataFrame(SMARTWEALTH_JSON["result"])
        df.drop(columns='type', inplace=True)
        df.rename(columns={'companyId': 'CompanyId', 'nepseCompanyId': 'NepseId',
                           'companyName': 'Company', 'stockSymbol': 'Symbol', 'sector': 'Sector'}, inplace=True)

    if save_to_csv:
        path = "./SmartWealthDataset"
        if not os.path.exists(path):
            os.makedirs(path)
        df.to_csv(path + '/smartwealthpro_company_list.csv', index=False)

    return df


def smart_wealth_company_history(companyId: str, startDate: str = "", endDate: str = "", save_to_csv=False) -> pd.DataFrame:
    """ Fetch Company History from SmartWealthPro.

    Args:
        companyId: (str) - Ex : 154
        startDate: (str) - Ex : "2010-01-01" (YYYY-MM-DD)
        endDate: (str) - Ex : "2020-01-01"  (YYYY-MM-DD)
        save_to_csv: (bool) - Ex : True

    Returns:
        df: (pd.DataFrame)
    """
    #
    url = "https://app.smartwealthpro.com/api/GetDailyHistoricalData?type=stock&id="+companyId+"&fromDate=" + \
        startDate+"&toDate="+endDate + \
        "&pageNo=1&itemsPerPage=9000000&pagePerDisplay=5&_=1648522274261"
    payload = {}
    headers = {
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Microsoft Edge";v="99"',
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Content-Type': 'application/json; charset=utf-8',
        'X-Requested-With': 'XMLHttpRequest',
        'sec-ch-ua-mobile': '?0',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36 Edg/99.0.1150.55',
        'sec-ch-ua-platform': '"Windows"',
        'Cookie': '.AspNetCore.Antiforgery.c2MYB_mZqSE=CfDJ8HpU54MlOTxHvXVwP3W4FVcE04C__HY9Ao_d2SGGKq0B4N-qC9sLKRJtbZgK6GcBsxbVn836dm4Smfhp9Q3gN3xxVVM7biw_G-LDmFOmxIxvMZCBJFgCtTHT-Ndk6_0YtyL4tlMZ8oMM9E_3yJdG_Pg; .AspNetCore.Identity.Application=CfDJ8HpU54MlOTxHvXVwP3W4FVfmqWzHboJ_XA8aYLA0dG047x_SBwgmrSDqUXI8WCVdNor3x6hRkzgZsr1Y62Xg6rcURZQ2J-0DwtwaENsDAey7QPUxmr9h1Jw6zidjV5YEktdvP49Q6UIy7_5MjV-uT04OL0k1TsjCaMTNvNGCBBbPVjmGQ_r4a8GH-LT0ifqNZ3TzVBugIQFonBpjk6RtxLkHdqmgDy2b-vUjq4Zn1JVqEOna_devKeiYR5Ev-rteUUTaO7G3ebX4lTTohvfaAP8M2BwgXhE2xkKeWz1PcCA7izJTfQ0jNXdZ7C_lG_L8ybYsPBu2ZeTESjHPwqIY1XLlf4MPipSbktORRD76ZCGuU_k1uKWXa1AwqiNxvIh-Pi9svDFCTa-jLtJk-yKrX5bcQy23wcUnWcHCpA1IJmg1_iUAKw3oRsM26R19cviC0FhuyWI1BXcdV2hOHPlqFz5M4h3A21IjDsaCzTRwZqG_NrE69k9iAnfPMD_2VX3HhXZ8DE3fwjwaIHqKR7sPYdBf6_5y-AH8ctfohcMZAYdiv42yVgfAlAuYry5mp9UipNQOstAVO2qPBD4Ie8LMiqgnQmX-uJDdwCJtWrubjKSkmisJt54Q0gxxGsO_PCPKamQ-mhfheDxyDhkoSLllKbkeSrVlv7dPeEBCLtS0LIFMF2jsH9aPSO3Ee4SDqMDRmEOw401nKYN-txm5oTXjHZhraL2kFGIvbn3-I4dIGDO9xxCuvme3GmQSQ22zZIfeiL3a7A61i1zr0lzquo4VbtQCiovJu1qgrxr623mYTOlcPyVhy_8OiZ0lbYGjmEHQsg'
    }

    response = requests.request("GET", url, headers=headers, data=payload)
    SMARTWEALTH_JSON = json.loads(response.text)
    if SMARTWEALTH_JSON["responseCode"] == 200:
        df = pd.DataFrame(SMARTWEALTH_JSON["result"]['data'])
        df.drop(columns='sNo', inplace=True)
        df.rename(columns={'tradeDate': 'Date', 'open': 'Open',
                  'high': 'High', 'low': 'Low', 'close': 'Close'}, inplace=True)
        df.insert(
            0, 'Symbol', SMARTWEALTH_JSON["result"]['summary']['stockSymbol'])

    if save_to_csv:
        path = "./SmartWealthDataset/Company"
        if not os.path.exists(path):
            os.makedirs(path)
        df.to_csv(path+'/smartwealthpro_' +
                  SMARTWEALTH_JSON["result"]['summary']['stockSymbol']+'_history.csv', index=False)

    return df


def smart_wealth_company_code(symbol: str) -> str:
    """
    Returns CompanyCode as per SmartWealthPro using stock symbol.

    Args:
        symbol: (str) - Ex : "AHPC"
    Returns:
        companyCode: (str) - Ex : "154"
    """
    cvs_file = pd.read_csv(
        './SmartWealthDataset/smartwealthpro_company_list.csv')
    return cvs_file[cvs_file['Symbol'] == symbol]['CompanyId'].values[0].__str__()


### **Data Mining [Extraction] ⛏**

Fetched top 50 list of companies listed on [Nepse](http://www.nepalstock.com) are shown and others are stored on csv file respectively.

In [6]:
nepse_company_names(save_to_csv=True).head()

,Stock Name,Stock Symbol,Sector,Symbol No
0,10 % NMB DEBENTURE 2085,NMBD2085,Corporate Debenture,2850
1,10% Himalayan Bank Debenture 2083,HBLD83,Corporate Debenture,2873
2,10% Laxmi Bank Debenture 2086,LBLD86,Corporate Debenture,2879
3,10% Nabil Debenture 2082,NBLD82,Corporate Debenture,2892
4,10% Nepal SBI Bank Debenture 2086,SBIBD86,Corporate Debenture,2890


Fetching[ SmartWealthPro's](#data-extraction-from-smartwealthprohttpsappsmartwealthprocom-website-📃) Company List with company code and extracting history of selected companies over interval.

In [7]:
selected_company_symbol = ["SHPC", "NABIL", "NLIC", "SICL", "MNBBL", "CIT"]
starting_date = "2000-01-01"
ending_date = "2022-12-31"

In [8]:
smart_wealth_company_list(save_to_csv=True).head()

,CompanyId,NepseId,Company,Symbol,Sector
0,596,2790,Aarambha Chautari Laghubitta Bittiya Sanstha L...,ACLBSL,Microfinance
1,259,2845,Adhikhola Laghubitta Bittiya Sanstha Limited,AKBSL,Microfinance
2,2,397,Agriculture Development Bank Limited,ADBL,Commercial Banks
3,583,2893,Ajod Insurance Limited,AIL,Non Life Insurance
4,151,2788,Ankhu Khola Jalvidhyut Company Ltd,AKJCL,Hydro Power


In [9]:
for company in selected_company_symbol:
    smart_wealth_company_history(companyId=smart_wealth_company_code(company),startDate=starting_date,endDate=ending_date,save_to_csv=True)

In [10]:
pd.read_csv('./SmartWealthDataset/Company/smartwealthpro_CIT_history.csv').head()

,Symbol,Date,Open,High,Low,Close
0,CIT,2022-03-28,3150.0,3170.0,3100.0,3120.0
1,CIT,2022-03-27,3165.0,3190.0,3141.0,3146.0
2,CIT,2022-03-24,3200.0,3248.9,3170.0,3170.0
3,CIT,2022-03-23,3269.0,3297.0,3230.0,3235.0
4,CIT,2022-03-22,3220.0,3290.0,3220.0,3269.0


In [11]:
pd.read_csv(
    './SmartWealthDataset/Company/smartwealthpro_MNBBL_history.csv').head()


,Symbol,Date,Open,High,Low,Close
0,MNBBL,2022-03-28,496.2,502.8,489.0,490.0
1,MNBBL,2022-03-27,519.1,519.1,495.0,496.5
2,MNBBL,2022-03-24,505.0,518.0,505.0,509.0
3,MNBBL,2022-03-23,520.0,520.0,511.0,514.9
4,MNBBL,2022-03-22,516.0,525.0,516.0,520.1


In [12]:
pd.read_csv(
    './SmartWealthDataset/Company/smartwealthpro_NABIL_history.csv').head()


,Symbol,Date,Open,High,Low,Close
0,NABIL,2022-03-28,1029.0,1029.0,1010.0,1017.0
1,NABIL,2022-03-27,1055.0,1055.0,1017.0,1020.0
2,NABIL,2022-03-24,1057.0,1057.0,1036.0,1040.0
3,NABIL,2022-03-23,1060.0,1060.0,1043.0,1047.0
4,NABIL,2022-03-22,1060.0,1065.0,1048.1,1050.0


In [13]:
pd.read_csv(
    './SmartWealthDataset/Company/smartwealthpro_NLIC_history.csv').head()


,Symbol,Date,Open,High,Low,Close
0,NLIC,2022-03-28,1098.0,1098.0,1057.2,1063.0
1,NLIC,2022-03-27,1144.0,1161.0,1080.0,1085.0
2,NLIC,2022-03-24,1158.9,1159.0,1120.0,1122.0
3,NLIC,2022-03-23,1170.1,1175.0,1138.0,1141.0
4,NLIC,2022-03-22,1125.0,1179.0,1124.0,1167.0


In [14]:
pd.read_csv(
    './SmartWealthDataset/Company/smartwealthpro_SHPC_history.csv').head()


,Symbol,Date,Open,High,Low,Close
0,SHPC,2022-03-28,392.9,396.9,386.0,388.0
1,SHPC,2022-03-27,402.0,402.0,390.0,392.9
2,SHPC,2022-03-24,405.1,407.0,401.0,401.1
3,SHPC,2022-03-23,407.0,409.0,405.0,405.0
4,SHPC,2022-03-22,415.0,416.9,409.0,409.0


In [15]:
pd.read_csv(
    './SmartWealthDataset/Company/smartwealthpro_SICL_history.csv').head()


,Symbol,Date,Open,High,Low,Close
0,SICL,2022-03-28,1380.0,1380.0,1325.0,1332.0
1,SICL,2022-03-27,1382.0,1428.0,1334.0,1360.0
2,SICL,2022-03-24,1414.0,1414.0,1380.0,1382.0
3,SICL,2022-03-23,1412.8,1412.9,1385.0,1395.0
4,SICL,2022-03-22,1400.0,1420.0,1392.0,1397.2
